In [1]:
import json
import numpy as np
import pandas as pd
from string import digits 
import re
from itertools import groupby
import collections

In [2]:
review = []
with open("../yelp_dataset_challenge_academic_dataset/yelp_academic_dataset_review.json") as f:
    for r in f:
        review.append(json.loads(r))

In [3]:
business_ids = [r['business_id'] for r in review]

In [5]:
business_ids_tuples = zip(range(len(business_ids)),business_ids)

In [6]:
combined_review_dic = dict.fromkeys(list(set(business_ids)))
for tup in business_ids_tuples:
    if combined_review_dic[tup[1]] is None:
        combined_review_dic[tup[1]] = review[tup[0]]['text']
    else:
        combined_review_dic[tup[1]] = " ".join([combined_review_dic[tup[1]], review[tup[0]]['text']])

In [7]:
len(combined_review_dic)

60785

In [8]:
#Remove punctuation
for k in combined_review_dic.keys():
    combined_review_dic[k] = re.sub(r'[^\w\s]', ' ', combined_review_dic[k].strip().replace("\'", " ").replace('\n'," "))

In [9]:
#Remove numeric characters:
for k in combined_review_dic.keys():
    combined_review_dic[k] = filter(lambda c: not c.isdigit(), combined_review_dic[k])

In [10]:
#lower cap:
for k in combined_review_dic.keys():
    combined_review_dic[k] = combined_review_dic[k].lower()

In [11]:
combined_review_dic['vcNAWiLM4dR7D2nwwJ7nCA']

u'dr  goldberg offers everything i look for in a general practitioner   he s nice and easy to talk to without being patronizing  he s always on time in seeing his patients  he s affiliated with a top notch hospital  nyu  which my parents have explained to me is very important in case something happens and you need surgery  and you can get referrals to see specialists without having to see him first   really  what more do you need   i m sitting here trying to think of any complaints i have about him  but i m really drawing a blank  unfortunately  the frustration of being dr  goldberg s patient is a repeat of the experience i ve had with so many other doctors in nyc    good doctor  terrible staff   it seems that his staff simply never answers the phone   it usually takes  hours of repeated calling to get an answer   who has time for that or wants to deal with it   i have run into this problem with many other doctors and i just don t get it   you have office workers  you have patients wit

In [12]:
import csv
with open('../yelp_dataset_challenge_academic_dataset/4000-most-common-english-words-csv.csv') as f:
    stopwords = [row for row in f]
top100stop = stopwords[1:101]
new = []
for w in top100stop:
    new.append(w.strip().lower())
top100stop = new

In [13]:
#Switch to list:
combined_review_list_dic = {}
for k in combined_review_dic.keys():
    combined_review_list_dic[k] = combined_review_dic[k].split()

In [ ]:
combined_review_list_nostop_dic = {}
t = 1
for k in combined_review_list_dic.keys():
    combined_review_list_nostop_dic[k]= [item for item in combined_review_list_dic[k] if item not in top100stop]
    print t
    t += 1

In [21]:
#save
with open('review_combined_inlist.json', 'w') as fp:
    json.dump(combined_review_list_nostop_dic, fp)

In [3]:
#reload
with open('review_combined_inlist.json', 'r') as fp:
    combined_review_list_nostop_dic = json.load(fp)

In [6]:
import inflect
p = inflect.engine()

In [8]:
def singular(word):
    sing = p.singular_noun(word)
    if sing is False:
        return word
    else:
        return sing

In [5]:
additional_words = ['is','m','i','ain','isn','haven','hadn','weren','t','s','ve','said','was']

In [6]:
combined_review_list_nostop_dic2 = {}
for k in combined_review_list_nostop_dic.keys():
    combined_review_list_nostop_dic2[k]= [item for item in combined_review_list_nostop_dic[k] if item not in additional_words]

In [ ]:
#save
with open('temp/review_combined_inlist2.json', 'w') as fp:
    json.dump(combined_review_list_nostop_dic2, fp)

In [3]:
#reload
with open('temp/review_combined_inlist2.json', 'r') as fp:
    combined_review_list_nostop_dic2 = json.load(fp)

In [4]:
res = np.load('temp/restaurants_index.npy')

In [14]:
restaurant_reviews = {}
for r in res:
    restaurant_reviews[r] =  combined_review_list_nostop_dic2[r]

In [15]:
word_list = []
for k in restaurant_reviews.keys():
    word_list += restaurant_reviews[k]

In [16]:
unique_words = list(set(word_list))

In [17]:
word_to_idx = dict(zip(unique_words,range(len(unique_words))))

In [18]:
with open('temp/word_to_idx_map.json', 'w') as fp:
    json.dump(word_to_idx, fp)

In [19]:
del word_list
del unique_words

In [21]:
venue_to_idx = dict(zip(restaurant_reviews.keys(),range(len(restaurant_reviews.keys()))))

In [22]:
with open('temp/venue_to_idx_map.json', 'w') as fp:
    json.dump(venue_to_idx, fp)

In [19]:
from scipy import sparse

nonzero_data = []
rows = []
cols = []

for k in restaurant_reviews.keys():
    counter = collections.Counter(restaurant_reviews[k])
    nonzero_data += counter.values()
    rows += [venue_to_idx[k]]*len(counter.values())
    cols += [word_to_idx[ck] for ck in counter.keys()]

sparse_res = sparse.csc_matrix((nonzero_data,(rows,cols)),shape = (len(venue_to_idx),len(word_to_idx)))

In [23]:
import scipy.io
scipy.io.mmwrite("temp/sparse_matrix_restaurants",sparse_res)

#Will be zipped

# Subsampling the data for computational reasons:

In [128]:
res_small = np.random.choice(res,504)
restaurant_reviews_small = {}
for r in res_small:
    restaurant_reviews_small[r] =  combined_review_list_nostop_dic2[r]
len(restaurant_reviews_small)

500

In [129]:
word_list_small = []
for k in restaurant_reviews_small.keys():
    word_list_small += restaurant_reviews_small[k]
unique_words_small = list(set(word_list_small))
word_to_idx_small = dict(zip(unique_words_small,range(len(unique_words_small))))
venue_to_idx_small = dict(zip(restaurant_reviews_small.keys(),range(len(restaurant_reviews_small.keys()))))

In [130]:
len(unique_words_small)

36745

In [131]:
from scipy import sparse

nonzero_data_small = []
rows_s = []
cols_s = []

for k in restaurant_reviews_small.keys():
    counter = collections.Counter(restaurant_reviews_small[k])
    nonzero_data_small += counter.values()
    rows_s += [venue_to_idx_small[k]]*len(counter.values())
    cols_s += [word_to_idx_small[ck] for ck in counter.keys()]

sparse_res_small = sparse.csc_matrix((nonzero_data_small,(rows_s,cols_s)),shape = (len(venue_to_idx_small),len(word_to_idx_small)))

In [135]:
import scipy.io
scipy.io.mmwrite("../cs281_data/sparse_matrix_restaurants_small",sparse_res_small)

In [136]:
with open('../cs281_data/word_to_idx_small_map.json', 'w') as fp:
    json.dump(word_to_idx_small, fp)
with open('../cs281_data/venue_to_idx_small_map.json', 'w') as fp:
    json.dump(venue_to_idx_small, fp)